# Building a Libor Curve

This is an example of a replication of a Quantlib example from

http://billiontrader.com/2015/02/16/bootstrapping-with-quantlib/

Agreement is very good however some issues about date generation need to be checked.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *

####################################################################
#  FINANCEPY BETA Version 0.350 - This build: 30 Apr 2024 at 22:32 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Creating the Libor Instruments

In [3]:
tradeDate = Date(16, 2, 2015)

### Load up the deposits first

In [4]:
spot_days = 0
settle_dt = tradeDate.add_weekdays(spot_days)

In [5]:
depoDCCType = DayCountTypes.ACT_360
depos = []

deposit_rate = 0.001375
depo = IborDeposit(settle_dt, "7D", deposit_rate, depoDCCType)
depos.append(depo)

deposit_rate = 0.001717
depo = IborDeposit(settle_dt, "1M", deposit_rate, depoDCCType)
depos.append(depo)

deposit_rate = 0.002112
depo = IborDeposit(settle_dt, "2M", deposit_rate, depoDCCType)
depos.append(depo)

deposit_rate = 0.002581
depo = IborDeposit(settle_dt, "3M", deposit_rate, depoDCCType)
depos.append(depo)

In [6]:
for depo in depos:
    print(depo)

OBJECT TYPE: IborDeposit
START DATE: 16-FEB-2015
MATURITY DATE: 23-FEB-2015
NOTIONAL: 100.0
DEPOSIT RATE: 0.001375
DAY COUNT TYPE: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.WEEKEND
BUS DAY ADJUST TYPE: BusDayAdjustTypes.MODIFIED_FOLLOWING

OBJECT TYPE: IborDeposit
START DATE: 16-FEB-2015
MATURITY DATE: 16-MAR-2015
NOTIONAL: 100.0
DEPOSIT RATE: 0.001717
DAY COUNT TYPE: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.WEEKEND
BUS DAY ADJUST TYPE: BusDayAdjustTypes.MODIFIED_FOLLOWING

OBJECT TYPE: IborDeposit
START DATE: 16-FEB-2015
MATURITY DATE: 16-APR-2015
NOTIONAL: 100.0
DEPOSIT RATE: 0.002112
DAY COUNT TYPE: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.WEEKEND
BUS DAY ADJUST TYPE: BusDayAdjustTypes.MODIFIED_FOLLOWING

OBJECT TYPE: IborDeposit
START DATE: 16-FEB-2015
MATURITY DATE: 18-MAY-2015
NOTIONAL: 100.0
DEPOSIT RATE: 0.002581
DAY COUNT TYPE: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.WEEKEND
BUS DAY ADJUST TYPE: BusDayAdjustTypes.MODIFIED_FOLLOWING



### Create Strips of Interest Rate Futures 

In [7]:
futs = []
fut = IborFuture(tradeDate, 1) ; futs.append(fut)
fut = IborFuture(tradeDate, 2) ; futs.append(fut)
fut = IborFuture(tradeDate, 3) ; futs.append(fut)
fut = IborFuture(tradeDate, 4) ; futs.append(fut)
fut = IborFuture(tradeDate, 5) ; futs.append(fut)
fut = IborFuture(tradeDate, 6) ; futs.append(fut)

### Convert Interest Rate Futures to FRAs

Need to supply futures price and the convexity (in percent) which is set to zero

In [8]:
fras = [None]*len(futs)
fras[0] = futs[0].to_fra(99.725,-0.0)
fras[1] = futs[1].to_fra(99.585,-0.00)
fras[2] = futs[2].to_fra(99.385,-0.00)
fras[3] = futs[3].to_fra(99.160,-0.00)
fras[4] = futs[4].to_fra(98.930,-0.00)
fras[5] = futs[5].to_fra(98.715,-0.00)

### Then we load up swap rates

In [9]:
accrual = DayCountTypes.ACT_360
freq = FrequencyTypes.ANNUAL
longEnd = DateGenRuleTypes.BACKWARD
swap_type = SwapTypes.PAY

In [10]:
spot_days = 2
settle_dt = tradeDate.add_weekdays(spot_days)

In [11]:
swaps = []
swap = IborSwap(settle_dt, "2Y", swap_type, 0.0089268, freq, accrual); swaps.append(swap)
swap = IborSwap(settle_dt, "3Y", swap_type, 0.0123343, freq, accrual); swaps.append(swap)
swap = IborSwap(settle_dt, "4Y", swap_type, 0.0147985, freq, accrual); swaps.append(swap)
swap = IborSwap(settle_dt, "5Y", swap_type, 0.0165843, freq, accrual); swaps.append(swap)

In [12]:
for swap in swaps:
    print(swap.termination_dt)

18-FEB-2017
18-FEB-2018
18-FEB-2019
18-FEB-2020


## Build the Curve

In [13]:
libor_curve = IborSingleCurve(tradeDate, depos, fras, swaps)

## Curve Examination

Generate a vector of time points and then generate zero rates and forward rates

In [14]:
for depo in depos:
    dt = depo.maturity_dt
    df = libor_curve.df(dt)
    zero_rate = libor_curve.zero_rate(dt, FrequencyTypes.SIMPLE, DayCountTypes.ACT_360)
    df = libor_curve.df(dt)
    print("%12s %12.8f %12.8f" % (dt, zero_rate*100.0, df))

 23-FEB-2015   0.13750000   0.99997326
 16-MAR-2015   0.17170000   0.99986647
 16-APR-2015   0.21120000   0.99965399
 18-MAY-2015   0.25810000   0.99934801


In [15]:
for fra in fras:
    dt = fra.maturity_dt
    df = libor_curve.df(dt)
    zero_rate = libor_curve.zero_rate(dt, FrequencyTypes.SIMPLE, DayCountTypes.ACT_360)
    print("%12s %12.8f %12.8f" % (dt, zero_rate*100.0, df))

 17-JUN-2015   0.25066108   0.99915821
 16-SEP-2015   0.32135287   0.99811116
 16-DEC-2015   0.40989345   0.99656192
 16-MAR-2016   0.50990211   0.99445037
 15-JUN-2016   0.61611302   0.99176792
 21-SEP-2016   0.73034320   0.98831075


In [16]:
for swap in swaps:
    dt = swap.termination_dt
    df = libor_curve.df(dt)
    zero_rate = libor_curve.zero_rate(dt, FrequencyTypes.ANNUAL, DayCountTypes.ACT_360)
    print("%12s %12.8f %12.8f" % (dt, zero_rate*100.0, df))

 18-FEB-2017   0.89065045   0.98210765
 18-FEB-2018   1.23669508   0.96320610
 18-FEB-2019   1.48896340   0.94170449
 18-FEB-2020   1.67274916   0.91921438


In [17]:
swaps[0].value(settle_dt, libor_curve, libor_curve)

-2.9760412871837616e-06

In [18]:
swaps[0].print_fixed_leg_pv()

START DATE: 18-FEB-2015
MATURITY DATE: 20-FEB-2017
COUPON (%): 0.89268
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
+---------+-------------+----------+--------+---------+--------+---------+---------+
| PAY_NUM |    PAY_dt   | NOTIONAL |  RATE  |   PMNT  |   DF   |    PV   |  CUM_PV |
+---------+-------------+----------+--------+---------+--------+---------+---------+
|    1    | 18-FEB-2016 | 1000000  | 0.8927 | 9050.78 | 0.9951 | 9006.29 | 9006.29 |
|    2    | 20-FEB-2017 | 1000000  | 0.8927 | 9125.17 | 0.982  | 8961.22 | 17967.5 |
+---------+-------------+----------+--------+---------+--------+---------+---------+


In [19]:
swaps[2].print_float_leg_pv()

START DATE: 18-FEB-2015
MATURITY DATE: 18-FEB-2019
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_E_360

PAYMENTS VALUATION:
+---------+-------------+----------+--------+---------+--------+---------+----------+
| PAY_NUM |    PAY_dt   | NOTIONAL |  IBOR  |   PMNT  |   DF   |    PV   |  CUM_PV  |
+---------+-------------+----------+--------+---------+--------+---------+----------+
|    1    | 18-MAY-2015 | 1000000  | 0.2579 |  644.78 | 0.9993 |  644.36 |  644.36  |
|    2    | 18-AUG-2015 | 1000000  | 0.3619 |  904.69 | 0.9984 |  903.29 | 1547.64  |
|    3    | 18-NOV-2015 | 1000000  | 0.5642 | 1410.54 | 0.997  | 1406.36 |  2954.0  |
|    4    | 18-FEB-2016 | 1000000  | 0.7888 | 1971.96 | 0.9951 | 1962.25 | 4916.25  |
|    5    | 18-MAY-2016 | 1000000  | 1.0007 | 2501.72 | 0.9926 | 2483.19 | 7399.44  |
|    6    | 18-AUG-2016 | 1000000  | 1.2463 | 3115.86 | 0.9895 | 3083.18 | 10482.62 |
|    7    | 18-NOV-2016 | 1000000  | 1.4595 | 3648.69 | 0.9859

Agreement with Quantlib example is very very good but not exact. Not sure how some of quantlib dates are generated - e.g. 19/2/2019 is a Tuesday so the 18th was a weekday and should have been used ?

Copyright (c) 2020 Dominic O'Kane